In [ ]:
import whisper
import json
import math
import librosa
import soundfile as sf
import noisereduce as nr
from pyannote.audio import Pipeline
from collections import defaultdict
import anthropic
import os

# =========================
# CONFIG
# =========================

AUDIO_PATH = "audio_input/001.mp3"
BASE_NAME = os.path.splitext(os.path.basename(AUDIO_PATH))[0] 
CLEAN_AUDIO_PATH = "convegno_clean.wav"
HF_TOKEN = 
ANTHROPIC_API_KEY = 
CLAUDE_MODEL = "claude-sonnet-4-6"

CHUNK_WORD_LIMIT = 4000

# =========================
# 1. NOISE REDUCTION
# =========================

print("Reducing noise...")
audio, sr = librosa.load(AUDIO_PATH, sr=None)
noise_sample = audio[0:5*sr]

reduced_noise = nr.reduce_noise(
    y=audio,
    sr=sr,
    y_noise=noise_sample,
    prop_decrease=0.8
)

sf.write(CLEAN_AUDIO_PATH, reduced_noise, sr)

# =========================
# 2. LOAD MODELS
# =========================

whisper_model = whisper.load_model("medium")

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    token=HF_TOKEN
)

claude_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# =========================
# 3. TRANSCRIPTION
# =========================

print("Transcribing...")
transcription = whisper_model.transcribe(
    CLEAN_AUDIO_PATH,
    language="it",
    word_timestamps=True
)

# =========================
# 4. DIARIZATION
# =========================

print("Running diarization...")
diarization = diarization_pipeline(CLEAN_AUDIO_PATH)

# =========================
# 5. MERGE TRANSCRIPT + SPEAKERS
# =========================

segments_output = []

diar_list = []

try:
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        diar_list.append((turn.start, turn.end, speaker))
    print("Metodo: itertracks")
except AttributeError:
    try:
        for row in diarization.segments:
            diar_list.append((row.start, row.end, row.speaker))
        print("Metodo: segments")
    except AttributeError:
        try:
            for row in diarization:
                diar_list.append((row.start, row.end, row.speaker))
            print("Metodo: iterabile diretto")
        except Exception as e:
            print(f"Nessun metodo funziona: {e}")
            print(f"Tipo oggetto: {type(diarization)}")
            print(f"Attributi: {[x for x in dir(diarization) if not x.startswith('_')]}")

print(f"Segmenti diarization trovati: {len(diar_list)}")
if diar_list:
    print(f"Esempio: {diar_list[0]}")

for segment in transcription["segments"]:
    seg_start = segment["start"]
    seg_end = segment["end"]
    seg_text = segment["text"].strip()

    speaker_label = "UNKNOWN"
    best_overlap = 0

    for (d_start, d_end, speaker) in diar_list:
        overlap = min(d_end, seg_end) - max(d_start, seg_start)
        if overlap > best_overlap:
            best_overlap = overlap
            speaker_label = speaker

    segments_output.append({
        "speaker": speaker_label,
        "start": seg_start,
        "end": seg_end,
        "text": seg_text
    })

# Merge segmenti consecutivi dello stesso speaker
merged_segments = []
current = segments_output[0]

for seg in segments_output[1:]:
    if seg["speaker"] == current["speaker"]:
        current["end"] = seg["end"]
        current["text"] += " " + seg["text"]
    else:
        merged_segments.append(current)
        current = seg

merged_segments.append(current)

print(f"Segmenti dopo merge: {len(merged_segments)}")

# =========================
# 6. IDENTIFY ROLES
# =========================

speaker_stats = defaultdict(lambda: {
    "total_duration": 0,
    "num_segments": 0,
    "total_words": 0
})

for seg in merged_segments:
    duration = seg["end"] - seg["start"]
    words = len(seg["text"].split())

    speaker_stats[seg["speaker"]]["total_duration"] += duration
    speaker_stats[seg["speaker"]]["num_segments"] += 1
    speaker_stats[seg["speaker"]]["total_words"] += words

sorted_speakers = sorted(
    speaker_stats.items(),
    key=lambda x: x[1]["total_duration"],
    reverse=True
)

role_mapping = {}

if sorted_speakers:
    role_mapping[sorted_speakers[0][0]] = "RELATORE_PRINCIPALE"

for speaker, stats in sorted_speakers[1:]:
    avg_words = stats["total_words"] / max(stats["num_segments"], 1)

    if stats["total_duration"] > 0.15 * sorted_speakers[0][1]["total_duration"]:
        role_mapping[speaker] = "RELATORE_SECONDARIO"
    elif avg_words < 40:
        role_mapping[speaker] = "PUBBLICO"
    else:
        role_mapping[speaker] = "INTERVENTO"

for seg in merged_segments:
    seg["role"] = role_mapping.get(seg["speaker"], "UNKNOWN")

# =========================
# 7. SAVE DIARIZED TRANSCRIPTION
# =========================

def format_timestamp(seconds):
    """Converte secondi in formato HH:MM:SS"""
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# Salva trascrizione diarizzata come TXT leggibile
print("Saving diarized transcription...")
with open(f"{BASE_NAME}trascrizione_diarizzata.txt", "w", encoding="utf-8") as f:
    f.write(f"TRASCRIZIONE DIARIZZATA\n")
    f.write(f"File: {AUDIO_PATH}\n")
    f.write("=" * 60 + "\n\n")
    for seg in merged_segments:
        ts_start = format_timestamp(seg["start"])
        ts_end = format_timestamp(seg["end"])
        f.write(f"[{ts_start} → {ts_end}]  {seg['role']} ({seg['speaker']})\n")
        f.write(f"{seg['text']}\n\n")

# Salva trascrizione diarizzata come JSON strutturato
with open(f"{BASE_NAME}trascrizione_diarizzata.json", "w", encoding="utf-8") as f:
    json.dump({
        "file_name": AUDIO_PATH,
        "num_speakers": len(speaker_stats),
        "speaker_stats": {
            spk: {
                "role": role_mapping.get(spk, "UNKNOWN"),
                "total_duration_sec": round(stats["total_duration"], 2),
                "num_segments": stats["num_segments"],
                "total_words": stats["total_words"]
            }
            for spk, stats in speaker_stats.items()
        },
        "segments": [
            {
                "speaker": seg["speaker"],
                "role": seg["role"],
                "start": round(seg["start"], 2),
                "end": round(seg["end"], 2),
                "duration": round(seg["end"] - seg["start"], 2),
                "text": seg["text"]
            }
            for seg in merged_segments
        ]
    }, f, indent=2, ensure_ascii=False)

print("Trascrizione diarizzata salvata: trascrizione_diarizzata.txt / .json")

# =========================
# 8. BUILD FULL TEXT FOR REGESTO
# =========================

full_text = ""
for seg in merged_segments:
    full_text += f"{seg['role']}: {seg['text']}\n\n"


# =========================
# 8b. CONTENT CLASSIFICATION
# =========================

CONTENT_TYPES = {
    "convegno": {
        "description": "Conferenza accademica, convegno, seminario, lezione universitaria",
        "system_prompt": "Sei un redattore accademico esperto nella redazione di regesti scientifici.",
        "chunk_prompt": """
Redigi un regesto analitico della seguente parte di convegno accademico.
Non distinguere per speaker. Usa linguaggio formale scientifico.
Evidenzia: tesi principale, argomentazioni, riferimenti bibliografici, conclusioni parziali.

{chunk}
""",
        "final_prompt": """
Integra i seguenti regesti parziali in un unico regesto coerente del convegno.
Struttura: 
- Tema centrale e obiettivi
- Argomentazioni principali sviluppate
- Riferimenti e fonti citate
- Conclusioni e prospettive di ricerca
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    },
    "dibattito": {
        "description": "Dibattito, intervista, talk show, confronto tra posizioni diverse",
        "system_prompt": "Sei un analista esperto nella sintesi di dibattiti e confronti dialettici.",
        "chunk_prompt": """
Redigi una sintesi analitica di questa parte di dibattito/intervista.
Identifica le posizioni espresse, senza attribuirle a singoli speaker.
Metti in evidenza: punti di accordo, punti di contrasto, argomenti chiave.

{chunk}
""",
        "final_prompt": """
Integra i seguenti estratti in un unico regesto coerente del dibattito.
Struttura:
- Tema e contesto del confronto
- Posizioni principali emerse
- Punti di tensione o disaccordo
- Elementi di convergenza o sintesi finale
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    },
    "discorso_politico": {
        "description": "Discorso istituzionale, comizio, intervento politico, cerimonia ufficiale",
        "system_prompt": "Sei un analista politico esperto nella sintesi di discorsi istituzionali.",
        "chunk_prompt": """
Redigi una sintesi di questa parte di discorso istituzionale/politico.
Identifica: messaggi chiave, riferimenti a politiche o eventi, tono e registro comunicativo.

{chunk}
""",
        "final_prompt": """
Integra i seguenti estratti in un unico regesto del discorso.
Struttura:
- Contesto e occasione
- Messaggi e temi centrali
- Riferimenti a fatti, dati o politiche
- Appelli e conclusioni
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    },
    "podcast_intervista": {
        "description": "Podcast informale, intervista giornalistica, conversazione divulgativa",
        "system_prompt": "Sei un redattore editoriale esperto nella sintesi di contenuti divulgativi.",
        "chunk_prompt": """
Riassumi questa parte di podcast/intervista in modo chiaro e accessibile.
Metti in evidenza: argomenti trattati, aneddoti rilevanti, informazioni principali condivise.

{chunk}
""",
        "final_prompt": """
Integra i seguenti estratti in un unico regesto del podcast/intervista.
Struttura:
- Argomento principale e contesto
- Temi sviluppati nel corso della conversazione
- Informazioni, storie o esempi notevoli
- Messaggi o takeaway finali
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    },
    "musica": {
        "description": "Contenuto musicale, concerto, performance, audio con prevalenza musicale",
        "system_prompt": "Sei un critico musicale esperto nella descrizione di performance e opere musicali.",
        "chunk_prompt": """
Descrivi questa sezione dell'audio musicale.
Identifica: genere, atmosfera, struttura percepibile, elementi vocali o strumentali rilevanti.

{chunk}
""",
        "final_prompt": """
Integra le seguenti descrizioni in un unico regesto della performance/opera musicale.
Struttura:
- Genere e carattere generale
- Struttura e fasi della performance
- Elementi distintivi (voci, strumenti, testi se presenti)
- Valutazione complessiva dell'opera
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    },
    "altro": {
        "description": "Contenuto non classificabile nelle categorie precedenti",
        "system_prompt": "Sei un redattore esperto nella sintesi di contenuti audio.",
        "chunk_prompt": """
Redigi una sintesi analitica della seguente parte di audio.
Linguaggio formale. Evidenzia i contenuti principali espressi.

{chunk}
""",
        "final_prompt": """
Integra i seguenti estratti in un unico regesto coerente.
Evita ripetizioni. Nessun elenco puntato. Massimo 300 parole.

{combined}
"""
    }
}


def classify_content(text_sample: str, client: anthropic.Anthropic, model: str) -> tuple[str, str]:
    """
    Classifica il tipo di contenuto audio analizzando un campione di testo.
    Restituisce (categoria, motivazione).
    """
    categories_desc = "\n".join([
        f"- '{k}': {v['description']}"
        for k, v in CONTENT_TYPES.items()
    ])

    # Usa solo i primi 3000 parole come campione per la classificazione
    sample = " ".join(text_sample.split()[:3000])

    response = client.messages.create(
        model=model,
        max_tokens=300,
        temperature=0,
        system="Sei un classificatore di contenuti audio. Rispondi SOLO in formato JSON.",
        messages=[
            {
                "role": "user",
                "content": f"""
Analizza il seguente testo trascritto da un audio e classificalo in una delle categorie seguenti:

{categories_desc}

Testo:
{sample}

Rispondi ESCLUSIVAMENTE con questo JSON (nessun testo aggiuntivo):
{{
  "categoria": "<nome_categoria>",
  "confidenza": <valore_da_0_a_1>,
  "motivazione": "<breve spiegazione>"
}}
"""
            }
        ]
    )

    import re
    raw = response.content[0].text.strip()

    # Estrai JSON anche se c'è testo attorno
    match = re.search(r'\{.*\}', raw, re.DOTALL)
    if match:
        result = json.loads(match.group())
    else:
        result = {"categoria": "altro", "confidenza": 0.0, "motivazione": "Parsing fallito"}

    categoria = result.get("categoria", "altro")
    if categoria not in CONTENT_TYPES:
        categoria = "altro"

    motivazione = result.get("motivazione", "")
    confidenza = result.get("confidenza", 0.0)

    print(f"📂 Contenuto classificato come: '{categoria}' (confidenza: {confidenza:.0%})")
    print(f"   Motivazione: {motivazione}")

    return categoria, motivazione


# Esegui classificazione
print("Classifying content type...")
content_category, classification_reason = classify_content(full_text, claude_client, CLAUDE_MODEL)
template = CONTENT_TYPES[content_category]

# =========================
# 9. CHUNKING
# =========================

def split_chunks(text, limit):
    words = text.split()
    return [
        " ".join(words[i:i+limit])
        for i in range(0, len(words), limit)
    ]

chunks = split_chunks(full_text, CHUNK_WORD_LIMIT)

# =========================
# 10. PARTIAL REGESTI WITH CLAUDE (template-based)
# =========================

partial_regesti = []

for idx, chunk in enumerate(chunks):
    print(f"Processing chunk {idx+1}/{len(chunks)}")

    response = claude_client.messages.create(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        temperature=0.2,
        system=template["system_prompt"],
        messages=[
            {
                "role": "user",
                "content": template["chunk_prompt"].format(chunk=chunk)
            }
        ]
    )

    partial_text = response.content[0].text
    partial_regesti.append(partial_text)

# =========================
# 11. FINAL REGESTO (template-based)
# =========================

combined = "\n\n".join(partial_regesti)

final_response = claude_client.messages.create(
    model=CLAUDE_MODEL,
    max_tokens=4000,
    temperature=0.2,
    system=template["system_prompt"],
    messages=[
        {
            "role": "user",
            "content": template["final_prompt"].format(combined=combined)
        }
    ]
)

final_regesto = final_response.content[0].text

# =========================
# 12. SAVE REGESTO OUTPUT
# =========================

with open(f"{BASE_NAME}regesto_finale.txt", "w", encoding="utf-8") as f:
    f.write(final_regesto)

with open(f"{BASE_NAME}regesto_finale.json", "w", encoding="utf-8") as f:
    json.dump({
        "file_name": AUDIO_PATH,
        "regesto_finale": final_regesto
    }, f, indent=2, ensure_ascii=False)

# =========================
# 13. SUMMARY
# =========================

print("\n✅ Pipeline completa.")
print(f"   📄 trascrizione_diarizzata.txt  — trascrizione con speaker e timestamp")
print(f"   📋 trascrizione_diarizzata.json — trascrizione strutturata in JSON")
print(f"   📝 regesto_finale.txt           — regesto accademico")
print(f"   📦 regesto_finale.json          — regesto in JSON")

Reducing noise...
Transcribing...
Running diarization...


/home/dhlab/miniconda3/envs/digimab/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:103: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1857.)
  std = sequences.std(dim=-1, correction=1)


Nessun metodo funziona: 'DiarizeOutput' object is not iterable
Tipo oggetto: <class 'pyannote.audio.pipelines.speaker_diarization.DiarizeOutput'>
Attributi: ['exclusive_speaker_diarization', 'serialize', 'speaker_diarization', 'speaker_embeddings']
Segmenti diarization trovati: 0
Segmenti dopo merge: 1
Saving diarized transcription...
Trascrizione diarizzata salvata: trascrizione_diarizzata.txt / .json
Classifying content type...
📂 Contenuto classificato come: 'convegno' (confidenza: 95%)
   Motivazione: Il testo trascritto è chiaramente una riunione/convegno istituzionale e accademico dedicato alle celebrazioni del bicentenario leopardiano. Sono presenti più relatori (assessori, ambasciatori, rappresentanti di ministeri, professori universitari) che si alternano in interventi formali, si discutono programmi nazionali e internazionali, coordinamenti tra istituzioni, leggi di finanziamento e comitati scientifici. Il tono e la struttura corrispondono a un convegno/seminario istituzionale